#RAG and TruLens Evaluation - advanced techniques with sentence window retrieval

RAG - retrieval augmented generation - is an antidote for contemporary LLMs hallucinations. However, naive RAG is prone to making up facts and not leveraging the knowledge base it has access to. In this notebook we will present techniques for efficient managing available knowledge, advanced retrieval techniqes and reranking capabilities. We base our design on:
 - Llama_index's retrieval module utilizing sentence window retrieval technique for more efficient context retrieval,
 - Llama_index's default VectorStoreIndex, which uses an in-memory SimpleVectorStore that's initialized as part of the default storage context,
 - Llama2-13B as the LLM answer generator
 -MTEB's leading sentence embeddings (intfloat/e5-base-v2 and BAAI/bge-base-en-v1.5 were tested)
 - SentenceTransformers based reranker "BAAI/bge-reranker-base"

**Sentence window retrieval** technique is based on idea of retrieving a very granular piece of information and then enriching it with the context around it (which size is specified by the window size parameter). For instance, sentence window size 3 means roughly 3 sentences before and after given selected sentence.

Furthermore, evaluation of different RAG setups will be performed with the use TruLens RAG benchmark, which employs OpenAI as reasoning engine to evaluate generated outputs with respect to three main components (so called RAG Triad):
- Answer Relevance
- Context Relevance
- Groundedness

For this exercise I will use 290 page pdf book "Building Knowledge Graphs - A Practitioner's Guide" by J. Barrasa and J. Webber (compliments of neo4j).


In [ ]:
!pip install -qU \
  transformers \
  sentence-transformers \
  accelerate \
  langchain \
  bitsandbytes \
  llama_index \
  trulens_eval \
  pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 632.1/632.1 kB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.9/277.9 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 7.1 M

Some examples of questions to ask:

1. Explain how identity graph improved consumer insight for Meredith Corporation
2. What is the claim meredith corporation is making wrt the identy graph methods they employed?
3. Explain how can knowledge graphs be employed to solve the entity matching problem
4. In entity resolution problems what are strong and weak identifier? Explain with examples from the text.
5. How can knowledge graphs be used in entity resolution problems where there are no strong identifiers
6. List steps to designing a record deduplication solution using identity graphs
7. What are the challenges of working with unstructured data in entity resolution problem and how can they be addressed with graph-based solution
8. What are the common use cases where deduplication of data appear?
9. How can you employ graph-based solution for fraud detection?
10. What is a fraud-ring pattern and what are ways to detect it?
11. What are some pitfalls when detecting fraud ring and how to avoid them?
12. Explain how can knowledge graphs help to match skillsets of employers in an organization with particular project's needs.

#1. Get Knowledge Graph Practicioner guide


In [ ]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["/content/drive/MyDrive/ML/data/RAG/Building-Knowledge-Graphs-Practitioner's-Guide-OReilly-book.pdf"]
).load_data()

print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))

<class 'list'> 

291 

<class 'llama_index.schema.Document'>


In [ ]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [ ]:
document.text[10000:20000]

'                   130\nMetadata Graph Example                                                                                             130\nQuerying the Metadata Graph Model                                                                         131\nUsing Relationships to Connect Data and Metadata                                                133\nSummary                                                                                                                         134\n9.Identity Knowledge Graphs. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .  135\nKnowing Y our Customer                                                                                              135\nWhen Does the Problem Appear?                                                                               136\nGraph-Based Entity Resolution Step by Step                                                             137\nData Preparation                     

In [ ]:
from llama_index.node_parser import SentenceWindowNodeParser

# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

nodes = node_parser.get_nodes_from_documents([document])
print(len(nodes), nodes[1000].metadata["window"])

4041 Labels don’t tell us, for example, that one category is broader than another one or that
certain products are compatible with others or are even possible substitutions based
on the categories to which they belong.  Y ou cannot tell from labels alone that giraffe
is a specialization of mammal  or that apple  could be a reasonable substitution for
banana  when you are interested in fruit .  This functionality is present in some other
graph models, notably in some implementations of the Semantic Web technology
stack, but the implementation complexity is high, and reasoning over the model is
often computationally slow, even intractable.
 At a business level, lacking data about categories and substitutions means lost sales
opportunities.  It is clear that better product catalogs mean better shopping experi‐
ences for the customer and more revenue for the retailer.  An expressive way of
categorizing products is therefore a worthwhile investment.



#3. Get LLama2-13B and Embeddings

##3.1. Llama2-13B and Embeddings, setup the service context

We load Llama2-13B, fitting it on a single 15GB RAM with the help of bitsandbytes quantization.

In [ ]:
%%time
from torch import cuda, bfloat16
import transformers
import torch


model_name = "meta-llama/Llama-2-13b-chat-hf"
# set quantization configuration using bitsandbytes lib
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)


hf_auth = os.environ.get('HF_API_KEY')
model_config = transformers.AutoConfig.from_pretrained(
    model_name,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:1067: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

CPU times: user 42.4 s, sys: 50.2 s, total: 1min 32s
Wall time: 3min 50s


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRM

Let's select some decent embeddings model based on [HF's leaderboard](https://huggingface.co/spaces/mteb/leaderboard).

In [ ]:
%%time
import torch
from langchain.llms.base import LLM
from llama_index.prompts import PromptTemplate

# This will wrap the default prompts that are internal to llama-index
# taken from https://huggingface.co/Writer/camel-5b-hf
query_wrapper_prompt = PromptTemplate(
    "Below is an instruction that describes a task. "
    "Write a response that appropriately completes the request.\n\n"
    "### Instruction:\n{query_str}\n\n### Response:"
)

model_name = "meta-llama/Llama-2-13b-chat-hf"
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name,
    use_auth_token=hf_auth
)


llm = HuggingFaceLLM(
    model=model,
    tokenizer=tokenizer,
    context_window=2048,
    max_new_tokens=512,
    generate_kwargs={
        "temperature": 0.01,
        "repetition_penalty": 1.1 },
    query_wrapper_prompt=query_wrapper_prompt
)


from llama_index import ServiceContext
from llama_index.embeddings import HuggingFaceEmbedding


# loads 'intfloat/e5-base-v2'
embed_model = HuggingFaceEmbedding(model_name='intfloat/e5-base-v2')

sentence_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
    node_parser=node_parser,
)

service_context = ServiceContext.from_defaults(chunk_size=512, llm=llm)

CPU times: user 563 ms, sys: 330 ms, total: 893 ms
Wall time: 745 ms


### 3.2 Build the index

In [ ]:
from llama_index import VectorStoreIndex

sentence_index = VectorStoreIndex.from_documents(
    [document], service_context=sentence_context
)

In [ ]:
sentence_index.storage_context.persist(persist_dir="/content/drive/MyDrive/ML/data/RAG/sentence_index")


In [ ]:
# This block of code is optional to check
# if an index file exist, then it will load it
# if not, it will rebuild it

import os
from llama_index import VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index import load_index_from_storage

if not os.path.exists("/content/drive/MyDrive/ML/data/RAG/sentence_index"):
    sentence_index = VectorStoreIndex.from_documents(
        [document], service_context=sentence_context
    )

    sentence_index.storage_context.persist(persist_dir="/content/drive/MyDrive/ML/data/RAG/sentence_index")
else:
    sentence_index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir="/content/drive/MyDrive/ML/data/RAG/sentence_index"),
        service_context=sentence_context
    )

###3.3. Build the post processor


In [ ]:
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.schema import NodeWithScore
from copy import deepcopy

postproc = MetadataReplacementPostProcessor(
    target_metadata_key="window"
)
scored_nodes = [NodeWithScore(node=x, score=1.0) for x in nodes]
nodes_old = [deepcopy(n) for n in nodes]
replaced_nodes = postproc.postprocess_nodes(scored_nodes)

In [ ]:
len(scored_nodes)

4041

In [ ]:
print(nodes_old[1000].text)

At a business level, lacking data about categories and substitutions means lost sales
opportunities. 


In [ ]:
print(replaced_nodes[1000].text)

Labels don’t tell us, for example, that one category is broader than another one or that
certain products are compatible with others or are even possible substitutions based
on the categories to which they belong.  Y ou cannot tell from labels alone that giraffe
is a specialization of mammal  or that apple  could be a reasonable substitution for
banana  when you are interested in fruit .  This functionality is present in some other
graph models, notably in some implementations of the Semantic Web technology
stack, but the implementation complexity is high, and reasoning over the model is
often computationally slow, even intractable.
 At a business level, lacking data about categories and substitutions means lost sales
opportunities.  It is clear that better product catalogs mean better shopping experi‐
ences for the customer and more revenue for the retailer.  An expressive way of
categorizing products is therefore a worthwhile investment.



###3.4. Add reranker

In [ ]:
from llama_index.indices.postprocessor import SentenceTransformerRerank

# BAAI/bge-reranker-base
# link: https://huggingface.co/BAAI/bge-reranker-base
rerank = SentenceTransformerRerank(
    top_n=2, model="BAAI/bge-reranker-base"
)

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [ ]:
from llama_index import QueryBundle
from llama_index.schema import TextNode, NodeWithScore

query = QueryBundle("I want a dog.")

scored_nodes = [
    NodeWithScore(node=TextNode(text="This is a cat"), score=0.6),
    NodeWithScore(node=TextNode(text="This is a dog"), score=0.4),
]

In [ ]:
reranked_nodes = rerank.postprocess_nodes(
    scored_nodes, query_bundle=query
)
print([(x.text, x.score) for x in reranked_nodes])

[('This is a dog', 0.91827416), ('This is a cat', 0.001404082)]


In [ ]:
sentence_window_engine = sentence_index.as_query_engine(
    top_n=3, similarity_top_k=6, node_postprocessors=[postproc, rerank]
)

#4. Testing the Sentence-Window retriever setup



In [ ]:
window_response = sentence_window_engine.query(
    "How can knowledge graphs be used in entity resolution problems where there are no strong identifiers?"
)
display_response(window_response)

**`Final Response:`** To address entity resolution problems where there are no strong identifiers, knowledge graphs can be leveraged by utilizing their ability to capture complex relationships between entities. By integrating multiple sources of data, knowledge graphs can help identify entities with similar meanings or concepts, even in the absence of strong identifiers. This can be achieved through the use of semantic reasoning techniques, such as inference and deduction, which can help connect related entities across different datasets. Additionally, knowledge graphs can provide a framework for organizing and structuring data, making it easier to identify patterns and relationships that may not be immediately apparent. Overall, the use of knowledge graphs in entity resolution can help improve the accuracy and efficiency of the process, particularly in situations where there are no clear identifiers available.

In [ ]:
window_response = sentence_window_engine.query(
    "Explain how identity graph improved consumer insight for Meredith Corporation?"
)
display_response(window_response)

**`Final Response:`** The identity graph provided by the algorithm in Example 9-22 helped improve consumer insight for Meredith Corporation by enabling the company to match entities across different sources. In this case, the algorithm used product titles to find similarities between entities from different sources, such as Amazon and Google product catalogs. By merging these entities into a single graph, the algorithm was able to provide a more comprehensive view of consumer behavior and preferences. This allowed Meredith Corporation to better understand its audience and tailor its content to their needs, resulting in more personalized experiences for users. Additionally, the use of weak identifiers and non-identifying features, such as text descriptions, enabled the algorithm to capture more subtle patterns and relationships in the data, leading to more accurate and informative insights. Overall, the identity graph provided by the algorithm helped Meredith Corporation gain a more complete understanding of its consumers and their preferences, allowing the company to make more informed decisions about its content and marketing strategies.

In [ ]:
window_response = sentence_window_engine.query(
    "List steps to designing a record deduplication solution using identity graphs"
)
display_response(window_response)

**`Final Response:`** To design a record deduplication solution using identity graphs, follow these steps:

Step 1: Define the Master Entities
The first step is to identify the master entities in the data, which are the entities that have unique identifiers, such as person, organization, or location. These entities will serve as the foundation for the identity graph.

Step 2: Create the Identity Graph
Once the master entities have been identified, create an identity graph that connects them through relationships based on their unique identifiers. For example, if two people have the same name and date of birth, they can be connected through a relationship.

Step 3: Define the Relationships
Define the relationships between the master entities, such as one-to-one, one-to-many, or many-to-many. This will help identify duplicate records and determine the degree of similarity between them.

Step 4: Enrich the Data
Enrich the data by adding additional attributes to the master entities and relationships, such as addresses, phone numbers, or email addresses. This will provide more context and make it easier to identify duplicate records.

Step 5: Query the Graph
Use Cypher queries to query the identity graph and identify duplicate records. For example, you can match on the unique identifier of a person and return all the related records that have the same identifier.

Step 6: Merge the Records
Once duplicate records have been identified, merge them into a single record. This can be done by selecting the most up-to-date version of the record and discarding the duplicates.

Step 7: Validate the Results
Validate the results of the deduplication process to ensure that the correct records have been merged and that no errors have occurred during the process.

By following these steps, you can design an effective record deduplication solution using identity graphs.

Let's create a function to encapsulate all the above steps:

In [ ]:
import os
from torch import cuda, bfloat16
import transformers
import torch
from llama_index import ServiceContext, VectorStoreIndex, StorageContext
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index import load_index_from_storage
from llama_index.prompts import PromptTemplate
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.llms import HuggingFaceLLM

def init_rag_llms(model_name = "meta-llama/Llama-2-13b-chat-hf", embed_model_name='intfloat/e5-base-v2'):

    # set quantization configuration using bitsandbytes lib
    bnb_config = transformers.BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=bfloat16
    )


    hf_auth = os.environ.get('HF_API_KEY')
    model_config = transformers.AutoConfig.from_pretrained(
        model_name,
        use_auth_token=hf_auth
    )

    model = transformers.AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        config=model_config,
        quantization_config=bnb_config,
        device_map='auto',
        use_auth_token=hf_auth
    )
    model.eval()

    tokenizer = transformers.AutoTokenizer.from_pretrained(
        model_name,
        use_auth_token=hf_auth
    )

    # This will wrap the default prompts that are internal to llama-index
    # taken from https://huggingface.co/Writer/camel-5b-hf
    query_wrapper_prompt = PromptTemplate(
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{query_str}\n\n### Response:"
    )


    llm = HuggingFaceLLM(
        model=model,
        tokenizer=tokenizer,
        context_window=2048,
        max_new_tokens=512,
        generate_kwargs={
            "temperature": 0.01,
            "repetition_penalty": 1.1 },
        query_wrapper_prompt=query_wrapper_prompt
    )

    # loads 'intfloat/e5-base-v2'
    embed_model = HuggingFaceEmbedding(model_name=embed_model_name)
    return llm, embed_model


def build_sentence_window_index(
    documents,
    llm,
    embed_model,
    sentence_window_size=3,
    save_dir="sentence_index",
):
    # create the sentence window node parser w/ default settings
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=sentence_window_size,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser,
    )
    if not os.path.exists(save_dir):
        sentence_index = VectorStoreIndex.from_documents(
            documents, service_context=sentence_context
        )
        sentence_index.storage_context.persist(persist_dir=save_dir)
    else:
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=sentence_context,
        )

    return sentence_index


def get_sentence_window_query_engine(
    sentence_index, similarity_top_k=6, rerank_top_n=3, rerank_model="BAAI/bge-reranker-base"
):
    # define postprocessors
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model=rerank_model
    )

    sentence_window_engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
    )
    return sentence_window_engine

##5. TruLens Evaluation

We compose a set of questions based on the KG Guide to test performance of our RAG against RAG Triad categories.



In [ ]:
eval_questions = ["Explain how identity graph improved consumer insight for Meredith Corporation",
"What is the claim meredith corporation is making wrt the identy graph methods they employed?",
"Explain how can knowledge graphs be employed to solve the entity matching problem",
"In entity resolution problems what are strong and weak identifier? Explain with examples from the text.",
"How can knowledge graphs be used in entity resolution problems where there are no strong identifiers",
"List steps to designing a record deduplication solution using identity graphs",
"What are the challenges of working with unstructured data in entity resolution problem and how can they be addressed with graph-based solution",
"What are the common use cases where deduplication of data appear?",
"How can you employ graph-based solution for fraud detection?",
"What is a fraud-ring pattern and what are ways to detect it?",
"What are some pitfalls when detecting fraud ring and how to avoid them?",
"Explain how can knowledge graphs help to match skillsets of employers in an organization with particular project's needs"]


In [ ]:
llm, embed_model = init_rag_llms(embed_model_name='BAAI/bge-base-en-v1.5')

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:1067: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:690: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
from trulens_eval.feedback import Groundedness
from llama_index.response.notebook_utils import display_response

from trulens_eval import (
    Feedback,
    TruLlama,
    OpenAI
)
from trulens_eval import Tru

def run_evals(eval_questions, tru_recorder, query_engine):
    for question in eval_questions:
        with tru_recorder as recording:
            response = query_engine.query(question)


def get_prebuilt_trulens_recorder(query_engine, app_id):
    openai = OpenAI()

    qa_relevance = (
        Feedback(openai.relevance_with_cot_reasons, name="Answer Relevance")
        .on_input_output()
    )

    qs_relevance = (
        Feedback(openai.relevance_with_cot_reasons, name = "Context Relevance")
        .on_input()
        .on(TruLlama.select_source_nodes().node.text)
        .aggregate(np.mean)
    )

    grounded = Groundedness(groundedness_provider=openai)

    groundedness = (
        Feedback(grounded.groundedness_measure_with_cot_reasons, name="Groundedness")
            .on(TruLlama.select_source_nodes().node.text)
            .on_output()
            .aggregate(grounded.grounded_statements_aggregator)
    )

    feedbacks = [qa_relevance, qs_relevance, groundedness]
    tru_recorder = TruLlama(
        query_engine,
        app_id=app_id,
        feedbacks=feedbacks
    )
    return tru_recorder

In [ ]:
from trulens_eval import Tru
import numpy as np

Tru().reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


##**5.1 Sentence window size = 3**

In [ ]:
tru_recorder_3 = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id='sentence window engine 3'
)

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [ ]:
run_evals(eval_questions, tru_recorder_3, sentence_window_engine)

In [ ]:
# Tru().run_dashboard()

In [ ]:
Tru().get_records_and_feedback(app_ids=[])[0]

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,sentence window engine 3,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_752dceda77dfc1b2a7c5d7ad98bad64b,"""Explain how identity graph improved consumer ...","""\nBased on the provided context information, ...",-,"{""record_id"": ""record_hash_752dceda77dfc1b2a7c...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T07:40:46.571306"", ""...",2023-12-21T07:41:29.114762,1.0,0.0,0.066667,[{'args': {'prompt': 'Explain how identity gra...,[{'args': {'prompt': 'Explain how identity gra...,[{'args': {'source': 'In a production deployme...,42,0,0.0
1,sentence window engine 3,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_3bd92513f62f210b6edca5ff67523936,"""What is the claim meredith corporation is mak...","""\nBased on the provided context information, ...",-,"{""record_id"": ""record_hash_3bd92513f62f210b6ed...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T07:41:30.040293"", ""...",2023-12-21T07:41:49.824698,1.0,0.0,0.425000,[{'args': {'prompt': 'What is the claim meredi...,[{'args': {'prompt': 'What is the claim meredi...,[{'args': {'source': 'The Cypher script in Exa...,19,0,0.0
2,sentence window engine 3,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_816d3b00232c5593faefca08f4cda66b,"""Explain how can knowledge graphs be employed ...","""\nKnowledge graphs can be used to address the...",-,"{""record_id"": ""record_hash_816d3b00232c5593fae...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T07:41:50.690512"", ""...",2023-12-21T07:42:15.133739,1.0,0.0,0.640000,[{'args': {'prompt': 'Explain how can knowledg...,[{'args': {'prompt': 'Explain how can knowledg...,[{'args': {'source': 'Navigating the Connectio...,24,0,0.0
3,sentence window engine 3,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_f8d78f8cd8486540394ea155d84a5b58,"""In entity resolution problems what are strong...","""\nIn entity resolution problems, strong ident...",-,"{""record_id"": ""record_hash_f8d78f8cd8486540394...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T07:42:15.994771"", ""...",2023-12-21T07:42:42.843346,1.0,0.0,0.200000,[{'args': {'prompt': 'In entity resolution pro...,[{'args': {'prompt': 'In entity resolution pro...,[{'args': {'source': 'The set of categories wi...,26,0,0.0
4,sentence window engine 3,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_2cad33c2a3ff93019ebf1183274886f9,"""How can knowledge graphs be used in entity re...","""\nKnowledge graphs can be utilized in entity ...",-,"{""record_id"": ""record_hash_2cad33c2a3ff93019eb...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T07:42:43.704150"", ""...",2023-12-21T07:43:04.729594,0.9,0.0,0.200000,[{'args': {'prompt': 'How can knowledge graphs...,[{'args': {'prompt': 'How can knowledge graphs...,[{'args': {'source': 'Example 13-37. Calculat...,21,0,0.0
5,sentence window engine 3,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_2e4e8e07bc04b87dae6f896ec640a64d,"""List steps to designing a record deduplicatio...","""\nTo design a record deduplication solution u...",-,"{""record_id"": ""record_hash_2e4e8e07bc04b87dae6...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T07:43:05.607337"", ""...",2023-12-21T07

In [ ]:
Tru().get_records_and_feedback(app_ids=[])[0].to_csv("/content/drive/MyDrive/ML/data/RAG/sentence_window_3.csv")

In [ ]:
import pandas as pd
sentence_window_3_pd = pd.read_csv("/content/drive/MyDrive/ML/data/RAG/sentence_window_3.csv")


In [ ]:
# get average Answer Relevance, Context Relevance and Groundedness
avg_ar = sentence_window_3_pd['Answer Relevance'].mean()
avg_cr = sentence_window_3_pd['Context Relevance'].mean()
avg_grd = sentence_window_3_pd['Groundedness'].mean()
(avg_ar, avg_cr, avg_grd)

(0.9916666666666667, 0.075, 0.2775474465148378)

Average Answer Relevance is high but that is just Llama's parametric knowledge answer. The context relevance and groundedness are very low with sentence window size = 3.


In [ ]:
Tru().get_records_and_feedback(app_ids=[])[0].iloc[8]

app_id                                              sentence window engine 3
app_json                   {"tru_class_info": {"name": "TruLlama", "modul...
type                       RetrieverQueryEngine(llama_index.query_engine....
record_id                       record_hash_23cc4c33bced8d43c5488ef5d4443f37
input                      "How can you employ graph-based solution for f...
output                     "\nTo employ a graph-based solution for fraud ...
tags                                                                       -
record_json                {"record_id": "record_hash_23cc4c33bced8d43c54...
cost_json                  {"n_requests": 0, "n_successful_requests": 0, ...
perf_json                  {"start_time": "2023-12-21T07:45:18.622988", "...
ts                                                2023-12-21T07:46:10.318039
Answer Relevance                                                         1.0
Context Relevance                                                        0.1

##**5.2. Sentence windows size: 5**

In [ ]:
sentence_window_size = 5
# embed_model1 = HuggingFaceEmbedding(model_name='BAAI/bge-base-en-v1.5')
sentence_index_5 = build_sentence_window_index(
    [document],
    llm=llm,
    embed_model=embed_model,
    sentence_window_size=sentence_window_size,
    save_dir=f"/content/drive/MyDrive/ML/data/RAG/sentence_index_{sentence_window_size}",
)
sentence_window_engine_5 = get_sentence_window_query_engine(
    sentence_index_5
)

Test on one question:


In [ ]:
window_response = sentence_window_engine_5.query(
    # "What is a fraud-ring pattern and what are ways to detect it?"
    "Explain how identity graph improved consumer insight for Meredith Corporation"
)
display_response(window_response)

**`Final Response:`** Based on the provided context information, I can explain how an identity graph improved consumer insight for Meredith Corporation.

Meredith Corporation is a media and marketing company that uses a knowledge graph to integrate data from various sources, including customer relationship management (CRM) systems, website analytics, and social media platforms. By integrating this data into a single, unified view, Meredith Corporation can gain a better understanding of its consumers and their behaviors.

One way the identity graph improves consumer insight is by providing a more complete and accurate view of individual consumers. By linking data from different sources to individual consumers, Meredith Corporation can gain a more comprehensive understanding of each consumer's preferences, interests, and behaviors. For example, by linking CRM data with website analytics, Meredith Corporation can see which consumers are most engaged with its content and which ones are most likely to convert into paying customers.

Another way the identity graph improves consumer insight is by enabling more effective segmentation and targeting. By grouping consumers based on shared characteristics and behaviors, Meredith Corporation can create more tailored and relevant marketing campaigns. For example, by identifying consumers who are most interested in health and wellness content, Meredith Corporation can create targeted campaigns that appeal to those consumers.

Finally, the identity graph enables Meredith Corporation to identify and address potential issues with its marketing efforts. By monitoring consumer behavior and sentiment in real-time, Meredith Corporation can quickly identify any issues or challenges and take corrective action. For example, if the identity graph reveals that a large number of consumers are experiencing difficulties with a particular product or service, Meredith Corporation can quickly address the issue and improve the overall consumer experience.

Overall, the identity graph provides Meredith Corporation with a powerful tool for gaining a deeper understanding of its consumers and improving its marketing efforts. By integrating data from multiple sources and providing a more complete and accurate view of individual consumers, the identity graph enables Meredith Corporation to create more effective marketing campaigns and improve the overall consumer experience.

In [ ]:
window_response = sentence_window_engine_5.query(
    # "What is a fraud-ring pattern and what are ways to detect it?"
    "How did Meredith Corporation consolidate their client profiles and by what number did they reduce duplication among them?"
)
display_response(window_response)

**`Final Response:`** Based on the provided context information, Meredith Corporation consolidated their client profiles using the WCC algorithm to identify unique sub-graphs within the larger graph. They incorporated more than 20 months of user data from both first- and third-party sources, resulting in over 4.4 terabytes of data for 30 billion nodes, 67 billion properties, and 35 billion relationships. The average length of touch points exploded from 14 days with a cookie to 241 days with user profiles, and average visits increased from 4 per cookie to 23.8 per profile. By consolidating nearly 350 million profiles into 163 million richer and more accurate profiles, Meredith was able to increase their understanding of their customers by 20 to 30%, leading to significant revenue gains and better-served consumers. The company reduced duplication among their client profiles by 163 million, representing a 35% reduction.

In [ ]:
tru_recorder_5 = get_prebuilt_trulens_recorder(
    sentence_window_engine_5,
    app_id='sentence window engine 5'
)

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [ ]:
run_evals(eval_questions, tru_recorder_5, sentence_window_engine_5)

In [ ]:
Tru().get_records_and_feedback(app_ids=[])[0]

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,sentence window engine 5,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_db215e32de065f3b3b84ffdc8eee277e,"""Explain how identity graph improved consumer ...","""\n\nBased on the provided context information...",-,"{""record_id"": ""record_hash_db215e32de065f3b3b8...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T22:10:56.806311"", ""...",2023-12-21T22:11:49.637736,1.0,0.000000,0.035714,[{'args': {'prompt': 'Explain how identity gra...,[{'args': {'prompt': 'Explain how identity gra...,[{'args': {'source': 'Example 5-11. Creating ...,52,0,0.0
1,sentence window engine 5,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_1771a205b212ddedd0d2ea0799e92b4a,"""What is the claim meredith corporation is mak...","""\nBased on the provided context information, ...",-,"{""record_id"": ""record_hash_1771a205b212ddedd0d...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T22:11:50.250063"", ""...",2023-12-21T22:12:09.198687,1.0,0.000000,0.325000,[{'args': {'prompt': 'What is the claim meredi...,[{'args': {'prompt': 'What is the claim meredi...,[{'args': {'source': 'The approach can be exte...,18,0,0.0
2,sentence window engine 5,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_7261a27f29af59c7377cee3111606497,"""Explain how can knowledge graphs be employed ...","""\n\nTo address the entity matching issue, kno...",-,"{""record_id"": ""record_hash_7261a27f29af59c7377...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T22:12:10.018276"", ""...",2023-12-21T22:12:39.592889,1.0,0.000000,0.375000,[{'args': {'prompt': 'Explain how can knowledg...,[{'args': {'prompt': 'Explain how can knowledg...,[{'args': {'source': 'Calculating the Wu and P...,29,0,0.0
3,sentence window engine 5,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_22a466fca8c520997e7687849e7675f4,"""In entity resolution problems what are strong...","""\nBased on the provided context information, ...",-,"{""record_id"": ""record_hash_22a466fca8c520997e7...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T22:12:40.197847"", ""...",2023-12-21T22:13:15.242898,1.0,0.000000,0.000000,[{'args': {'prompt': 'In entity resolution pro...,[{'args': {'prompt': 'In entity resolution pro...,[{'args': {'source': 'Is it a location? Is it...,35,0,0.0
4,sentence window engine 5,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_5f91cfa2ff31e0cd02d3817e79446eae,"""How can knowledge graphs be used in entity re...","""\nTo calculate the Wu and Palmer similarity m...",-,"{""record_id"": ""record_hash_5f91cfa2ff31e0cd02d...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T22:13:15.823596"", ""...",2023-12-21T22:14:13.400279,0.0,0.200000,0.587500,[{'args': {'prompt': 'How can knowledge graphs...,[{'args': {'prompt': 'How can knowledge graphs...,[{'args': {'source': 'Calculating the Wu and P...,57,0,0.0
5,sentence window engine 5,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_2bb0ca249945c0438204c9c6c71634bb,"""List steps to designing a record deduplicatio...","""\nTo design a record deduplication solution u...",-,"{""record_id"": ""record_hash_2bb0ca249945c043820...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T22:14:14.036629

In [ ]:
Tru().get_records_and_feedback(app_ids=[])[0].to_csv("/content/drive/MyDrive/ML/data/RAG/sentence_window_5.csv")

In [ ]:
# get average Answer Relevance, Context Relevance and Groundedness
import pandas as pd
sentence_window_5_pd = pd.read_csv("/content/drive/MyDrive/ML/data/RAG/sentence_window_5.csv")
# sentence_window_5_pd = sentence_window_5_pd.dropna(subset=['Answer Relevance'])
# sentence_window_5_pd = sentence_window_5_pd.reset_index(drop=True)
avg_ar = sentence_window_5_pd['Answer Relevance'].mean()
avg_cr = sentence_window_5_pd['Context Relevance'].mean()
avg_grd = sentence_window_5_pd['Groundedness'].mean()
(avg_ar, avg_cr, avg_grd)

(0.8363636363636363, 0.1303030303030303, 0.3364824722061564)

Try BGE-base embeddings to improve context relevance score.

In [ ]:
sentence_window_size = 5
embed_model1 = HuggingFaceEmbedding(model_name='BAAI/bge-base-en-v1.5')
sentence_index_5 = build_sentence_window_index(
    [document],
    llm=llm,
    embed_model=embed_model1,
    sentence_window_size=sentence_window_size,
    save_dir=f"/content/drive/MyDrive/ML/data/RAG/sentence_index_{sentence_window_size}",
)
sentence_window_engine_5 = get_sentence_window_query_engine(
    sentence_index_5
)

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
window_response = sentence_window_engine_5.query(
    # "What is a fraud-ring pattern and what are ways to detect it?"
    "How did Meredith Corporation consolidate their client profiles and by what number did they reduce duplication among them?"
)
display_response(window_response)

**`Final Response:`** Meredith Corporation consolidated their client profiles by utilizing a Knowledge Graph approach, specifically the Weisfeiler-Lehman (WCC) algorithm, to identify unique sub-graphs (connected components) within the larger graph. This allowed them to aggregate a set of weak identifiers into a strong identifier, resulting in a 20-30% increase in their understanding of their customers. By doing so, they were able to reduce duplication among their client profiles from nearly 350 million down to 163 million, resulting in more accurate and robust profiles.

In [ ]:
tru_recorder_5 = get_prebuilt_trulens_recorder(
    sentence_window_engine_5,
    app_id='sentence window engine 5'
)

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [ ]:
run_evals(eval_questions, tru_recorder_5, sentence_window_engine_5)

In [ ]:
Tru().get_records_and_feedback(app_ids=['sentence window engine 5'])[0]

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,sentence window engine 5,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_9a98fa9366c0890a905a4500ec39fafb,"""Explain how identity graph improved consumer ...","""\nBased on the provided context information, ...",-,"{""record_id"": ""record_hash_9a98fa9366c0890a905...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T22:54:55.190485"", ""...",2023-12-21T22:55:26.807519,1.0,0.600000,0.628571,[{'args': {'prompt': 'Explain how identity gra...,[{'args': {'prompt': 'Explain how identity gra...,[{'args': {'source': 'The average length of to...,31,0,0.0
1,sentence window engine 5,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_2652b58ea2c864ea1bc1645e6508348d,"""What is the claim meredith corporation is mak...","""\nBased on the provided context information, ...",-,"{""record_id"": ""record_hash_2652b58ea2c864ea1bc...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T22:55:27.336616"", ""...",2023-12-21T22:55:46.930854,1.0,0.600000,1.000000,[{'args': {'prompt': 'What is the claim meredi...,[{'args': {'prompt': 'What is the claim meredi...,[{'args': {'source': 'Matching using product t...,19,0,0.0
2,sentence window engine 5,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_d564aef24f925dd25c1223c9fd65247f,"""Explain how can knowledge graphs be employed ...","""\nKnowledge graphs can be utilized to address...",-,"{""record_id"": ""record_hash_d564aef24f925dd25c1...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T22:55:47.507771"", ""...",2023-12-21T22:56:08.727920,1.0,0.300000,1.000000,[{'args': {'prompt': 'Explain how can knowledg...,[{'args': {'prompt': 'Explain how can knowledg...,[{'args': {'source': 'Making the Annotation Se...,21,0,0.0
3,sentence window engine 5,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_b92a7b71b011eb34d9f4f9e17312cedf,"""In entity resolution problems what are strong...","""\nStrong and weak identifiers refer to the le...",-,"{""record_id"": ""record_hash_b92a7b71b011eb34d9f...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T22:56:09.514758"", ""...",2023-12-21T22:56:35.774556,1.0,0.433333,0.500000,[{'args': {'prompt': 'In entity resolution pro...,[{'args': {'prompt': 'In entity resolution pro...,"[{'args': {'source': 'To accommodate this, a p...",26,0,0.0
4,sentence window engine 5,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_53fb7b65f0c470c9c005792917be8e57,"""How can knowledge graphs be used in entity re...","""\nKnowledge graphs can be utilized in entity ...",-,"{""record_id"": ""record_hash_53fb7b65f0c470c9c00...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T22:56:36.379179"", ""...",2023-12-21T22:56:54.030269,1.0,0.300000,0.900000,[{'args': {'prompt': 'How can knowledge graphs...,[{'args': {'prompt': 'How can knowledge graphs...,[{'args': {'source': 'It created a virtuous cy...,17,0,0.0
5,sentence window engine 5,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_b97e2353c1efc8c3f92f3f7054326093,"""List steps to designing a record deduplicatio...","""\nTo design a record deduplication solution u...",-,"{""record_id"": ""record_hash_b97e2353c1efc8c3f92...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T22:56:54.59

In [ ]:
Tru().get_records_and_feedback(app_ids=['sentence window engine 5'])[0].to_csv("/content/drive/MyDrive/ML/data/RAG/sentence_window_5_bge.csv")

In [ ]:
# get average Answer Relevance, Context Relevance and Groundedness
import pandas as pd
sentence_window_5_pd = pd.read_csv("/content/drive/MyDrive/ML/data/RAG/sentence_window_5_bge.csv")
avg_ar = sentence_window_5_pd['Answer Relevance'].mean()
avg_cr = sentence_window_5_pd['Context Relevance'].mean()
avg_grd = sentence_window_5_pd['Groundedness'].mean()
(avg_ar, avg_cr, avg_grd)

(0.9916666666666667, 0.42424242424242425, 0.8297802197802199)

For the setup with window size=5 and embedding model of BAAI/bge-base-en-v1.5 RAG performs the best, which is reflected in TruLens' triad metric: mean Answer Relevance, Context Relevance and Groundedness.

##**5.3. Sentence window: 7**

In [ ]:
sentence_window_size = 7
# embed_model1 = HuggingFaceEmbedding(model_name='BAAI/bge-base-en-v1.5')
sentence_index_7 = build_sentence_window_index(
    [document],
    llm=llm,
    embed_model=embed_model1,
    sentence_window_size=sentence_window_size,
    save_dir=f"/content/drive/MyDrive/ML/data/RAG/sentence_index_{sentence_window_size}",
)
sentence_window_engine_7 = get_sentence_window_query_engine(
    sentence_index_7
)

In [ ]:
window_response = sentence_window_engine_7.query(
    # "What is a fraud-ring pattern and what are ways to detect it?"
    "Explain how identity graph improved consumer insight for Meredith Corporation"
)
display_response(window_response)

**`Final Response:`** To further enhance consumer insight for Meredith Corporation, the identity graph could be utilized to analyze the relationships between different data points. For instance, the graph could be queried to identify the correlation between purchase history and browsing habits, or the impact of social media engagement on subscription rates. Additionally, the graph could be used to identify key influencers within specific communities, allowing Meredith Corporation to collaborate with them to promote its brands and content.

By incorporating these advanced analytics capabilities, Meredith Corporation could gain even deeper insights into consumer behavior and preferences, enabling the company to make more informed decisions about its marketing strategies and content offerings.

In [ ]:
tru_recorder_7 = get_prebuilt_trulens_recorder(
    sentence_window_engine_7,
    app_id='sentence window engine 7'
)

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [ ]:
run_evals(eval_questions, tru_recorder_7, sentence_window_engine_7)

In [ ]:
Tru().get_records_and_feedback(app_ids=[])[0]

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,sentence window engine 7,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_7371ab2721dc89182d436e3e22fb99dd,"""Explain how identity graph improved consumer ...","""\n\nTo further enhance consumer insight for M...",-,"{""record_id"": ""record_hash_7371ab2721dc89182d4...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T22:32:43.868397"", ""...",2023-12-21T22:33:45.768383,1.0,0.0,0.050000,[{'args': {'prompt': 'Explain how identity gra...,[{'args': {'prompt': 'Explain how identity gra...,"[{'args': {'source': 'To do so, invoke the the...",61,0,0.0
1,sentence window engine 7,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_fc30186aa8b6e4ab27bbeefb2d123fe1,"""What is the claim meredith corporation is mak...","""\nBased on the provided context information, ...",-,"{""record_id"": ""record_hash_fc30186aa8b6e4ab27b...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T22:33:46.325773"", ""...",2023-12-21T22:34:06.124171,0.9,0.0,0.350000,[{'args': {'prompt': 'What is the claim meredi...,[{'args': {'prompt': 'What is the claim meredi...,[{'args': {'source': 'Example 10-3. Sampling ...,19,0,0.0
2,sentence window engine 7,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_af02a47b665105ba66fbe702082f64a1,"""Explain how can knowledge graphs be employed ...","""\nTo employ knowledge graphs in solving the e...",-,"{""record_id"": ""record_hash_af02a47b665105ba66f...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T22:34:06.918625"", ""...",2023-12-21T22:35:24.339337,1.0,0.0,0.340000,[{'args': {'prompt': 'Explain how can knowledg...,[{'args': {'prompt': 'Explain how can knowledg...,[{'args': {'source': 'While not as computation...,77,0,0.0
3,sentence window engine 7,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_aa4c16eca920dceb68c9026e2b78788b,"""In entity resolution problems what are strong...","""\nIn entity resolution problems, strong ident...",-,"{""record_id"": ""record_hash_aa4c16eca920dceb68c...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T22:35:24.937991"", ""...",2023-12-21T22:36:22.048155,0.9,0.0,0.666667,[{'args': {'prompt': 'In entity resolution pro...,[{'args': {'prompt': 'In entity resolution pro...,[{'args': {'source': '214 | Chapter 12: Semant...,57,0,0.0
4,sentence window engine 7,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_fc481e6804793578ccbda428560fa9aa,"""How can knowledge graphs be used in entity re...","""\nTo address entity resolution issues where t...",-,"{""record_id"": ""record_hash_fc481e6804793578ccb...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T22:36:22.623783"", ""...",2023-12-21T22:37:18.840175,1.0,0.0,0.760000,[{'args': {'prompt': 'How can knowledge graphs...,[{'args': {'prompt': 'How can knowledge graphs...,[{'args': {'source': 'Example 12-10. A Cypher...,56,0,0.0
5,sentence window engine 7,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_4da28b93b2cdc81bcd1372e6a572048a,"""List steps to designing a record deduplicatio...","""\nTo design a record deduplication solution u...",-,"{""record_id"": ""record_hash_4da28b93b2cdc81bcd1...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T22:37:19.383696"", ""...",2023-12-21T2

In [ ]:
Tru().get_records_and_feedback(app_ids=[])[0].to_csv("/content/drive/MyDrive/ML/data/RAG/sentence_window_7.csv")

In [ ]:
# get average Answer Relevance, Context Relevance and Groundedness
import pandas as pd
sentence_window_7_pd = pd.read_csv("/content/drive/MyDrive/ML/data/RAG/sentence_window_7.csv")
avg_ar = sentence_window_7_pd['Answer Relevance'].mean()
avg_cr = sentence_window_7_pd['Context Relevance'].mean()
avg_grd = sentence_window_7_pd['Groundedness'].mean()
(avg_ar, avg_cr, avg_grd)

(0.9166666666666666, 0.125, 0.5109459254196096)

Let's try with BGE-base embeddings:

In [ ]:
sentence_window_size = 7
sentence_index_7 = build_sentence_window_index(
    [document],
    llm=llm,
    embed_model=embed_model,
    sentence_window_size=sentence_window_size,
    save_dir=f"/content/drive/MyDrive/ML/data/RAG/sentence_index_{sentence_window_size}",
)
sentence_window_engine_7 = get_sentence_window_query_engine(
    sentence_index_7
)

In [ ]:
window_response = sentence_window_engine_7.query(
    # "What is a fraud-ring pattern and what are ways to detect it?"
    "Explain how identity graph improved consumer insight for Meredith Corporation"
)
display_response(window_response)

**`Final Response:`** Based on the provided context information, I can explain how an identity graph improved consumer insight for Meredith Corporation.

Meredith Corporation, a media and marketing company, incorporated an identity graph into their production system to better understand their customers. The identity graph integrated more than 20 months of user data from both first- and third-party sources, resulting in a database with over 4.4 terabytes of data for 30 billion nodes, 67 billion properties, and 35 billion relationships.

By leveraging the identity graph, Meredith was able to consolidate nearly 350 million profiles into 163 million richer and more accurate profiles, providing a high-definition view of user interests and preferences. This enabled stronger models, leading to more relevant content and more users returning over time, creating a virtuous cycle.

The identity graph allowed Meredith to increase their understanding of their customers by 20 to 30%, rather than just looking at individual cookies themselves. This improved consumer insight helped fuel stronger models, leading to more relevant content and more users returning over time.

In summary, the identity graph improved consumer insight for Meredith Corporation by providing a more comprehensive and accurate view of their customers, allowing them to better understand their interests and preferences, and fuel stronger models for more relevant content.

In [ ]:
tru_recorder_7 = get_prebuilt_trulens_recorder(
    sentence_window_engine_7,
    app_id='sentence window engine 7'
)

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [ ]:
run_evals(eval_questions, tru_recorder_7, sentence_window_engine_7)

In [ ]:
Tru().get_records_and_feedback(app_ids=[])[0]

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,sentence window engine 7,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_6fba6090e4a39908ca5404bc78638bd0,"""Explain how identity graph improved consumer ...","""\nBased on the provided context information, ...",-,"{""record_id"": ""record_hash_6fba6090e4a39908ca5...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T23:14:53.864266"", ""...",2023-12-21T23:15:29.151308,0.9,0.600000,0.875000,[{'args': {'prompt': 'Explain how identity gra...,[{'args': {'prompt': 'Explain how identity gra...,"[{'args': {'source': 'In production, the Mered...",35,0,0.0
1,sentence window engine 7,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_8bf3d9cb7782cb7e3a2393563b7233da,"""What is the claim meredith corporation is mak...","""\nBased on the provided context information, ...",-,"{""record_id"": ""record_hash_8bf3d9cb7782cb7e3a2...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T23:15:30.233922"", ""...",2023-12-21T23:15:52.074906,1.0,0.566667,1.000000,[{'args': {'prompt': 'What is the claim meredi...,[{'args': {'prompt': 'What is the claim meredi...,[{'args': {'source': 'The Cypher script in Exa...,21,0,0.0
2,sentence window engine 7,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_802347cd2d8eaa7223ded2ddddb8832f,"""Explain how can knowledge graphs be employed ...","""\nKnowledge graphs can be employed to solve t...",-,"{""record_id"": ""record_hash_802347cd2d8eaa7223d...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T23:15:52.660269"", ""...",2023-12-21T23:16:44.495505,1.0,0.066667,0.975000,[{'args': {'prompt': 'Explain how can knowledg...,[{'args': {'prompt': 'Explain how can knowledg...,"[{'args': {'source': 'One NASA engineer said, ...",51,0,0.0
3,sentence window engine 7,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_128c5ce49dd46be664349e934563896a,"""In entity resolution problems what are strong...","""\nIn entity resolution problems, strong and w...",-,"{""record_id"": ""record_hash_128c5ce49dd46be6643...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T23:16:45.087687"", ""...",2023-12-21T23:17:12.908810,1.0,0.333333,0.616667,[{'args': {'prompt': 'In entity resolution pro...,[{'args': {'prompt': 'In entity resolution pro...,[{'args': {'source': 'Entity Matching Matching...,27,0,0.0
4,sentence window engine 7,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_e68ceb20873ea7309f26fb8409acac35,"""How can knowledge graphs be used in entity re...","""\nKnowledge graphs can be utilized in entity ...",-,"{""record_id"": ""record_hash_e68ceb20873ea7309f2...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T23:17:13.475482"", ""...",2023-12-21T23:17:37.295896,1.0,0.300000,1.000000,[{'args': {'prompt': 'How can knowledge graphs...,[{'args': {'prompt': 'How can knowledge graphs...,[{'args': {'source': 'Entity Matching Matching...,23,0,0.0
5,sentence window engine 7,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_1d96c73204ecc025fa3689b6e8babb0c,"""List steps to designing a record deduplicatio...","""\nTo design a record deduplication solution u...",-,"{""record_id"": ""record_hash_1d96c73204ecc025fa3...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2023-12-21T23:17:37.

In [ ]:
Tru().get_records_and_feedback(app_ids=[])[0].to_csv("/content/drive/MyDrive/ML/data/RAG/sentence_window_7_bge.csv")

In [ ]:
# get average Answer Relevance, Context Relevance and Groundedness
import pandas as pd
sentence_window_7_pd = pd.read_csv("/content/drive/MyDrive/ML/data/RAG/sentence_window_7_bge.csv")
avg_ar = sentence_window_7_pd['Answer Relevance'].mean()
avg_cr = sentence_window_7_pd['Context Relevance'].mean()
avg_grd = sentence_window_7_pd['Groundedness'].mean()
(avg_ar, avg_cr, avg_grd)

(0.9818181818181819, 0.39999999999999997, 0.7587272727272727)

With growing sentence window size performance degrades, which is a sign of the model being overwhelmend with too much noisy context.

### Summary

For given exercise the best set of parameters wrt answer relevance, context relevance and groundedness are:
- Embedding model: BAAI/bge-base-en-v1.5
- Sentence window size: 5
- Best scores achieved: (answer relevance: 0.9916666666666667, context relevance: 0.42424242424242425, groundedness: 0.8297802197802199)
